In [1]:
import sys
import os

# Source: https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce

from lusi.ecoc import SVMRandomInvariantsECOC
from lusi.types import InvariantTypes

In [2]:
df = pd.read_csv("../data/ecoli.data", header=None)
df

,0,1,2,3,4,5,6,7,8
0,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
1,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
2,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
3,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
4,ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35,cp
...,...,...,...,...,...,...,...,...,...
331,TREA_ECOLI,0.74,0.56,0.48,0.5,0.47,0.68,0.30,pp
332,UGPB_ECOLI,0.71,0.57,0.48,0.5,0.48,0.35,0.32,pp
333,USHA_ECOLI,0.61,0.60,0.48,0.5,0.44,0.39,0.38,pp
334,XYLF_ECOLI,0.59,0.61,0.48,0.5,0.42,0.42,0.37,pp


In [3]:
X, y = df.iloc[:, :-1], df.iloc[:, -1].values
X

,0,1,2,3,4,5,6,7
0,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35
1,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44
2,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46
3,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36
4,ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35
...,...,...,...,...,...,...,...,...
331,TREA_ECOLI,0.74,0.56,0.48,0.5,0.47,0.68,0.30
332,UGPB_ECOLI,0.71,0.57,0.48,0.5,0.48,0.35,0.32
333,USHA_ECOLI,0.61,0.60,0.48,0.5,0.44,0.39,0.38
334,XYLF_ECOLI,0.59,0.61,0.48,0.5,0.42,0.42,0.37


In [4]:
# Transform labels to numerical values
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
hash_encoder = ce.HashingEncoder()
X = hash_encoder.fit_transform(X)
X = X.values
X

array([[0.  , 0.  , 0.  , ..., 0.56, 0.24, 0.35],
       [0.  , 0.  , 0.  , ..., 0.54, 0.35, 0.44],
       [1.  , 0.  , 0.  , ..., 0.49, 0.37, 0.46],
       ...,
       [1.  , 0.  , 0.  , ..., 0.44, 0.39, 0.38],
       [0.  , 0.  , 0.  , ..., 0.42, 0.42, 0.37],
       [0.  , 0.  , 0.  , ..., 0.31, 0.53, 0.52]])

In [6]:
num_dimensions = X.shape[1]
num_classes = len(np.unique(y))

print("Num dimensions: ", num_dimensions)
print("Num classes: ", num_classes)

Num dimensions:  15
Num classes:  8


In [7]:
def run_single_experiment(X_train, X_test, y_train, y_test, clf, model_parameters):
    model = GridSearchCV(clf, model_parameters, cv=5, scoring='accuracy', n_jobs=4)
    model.fit(X_train, y_train)
    
    print('Best estimator: ', model.best_estimator_)
    
    y_hat = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_hat)
    
    print('Accuracy: ', accuracy)
    
    return accuracy

## Experiment 1: Using 80% of the data

### Baseline model

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=47)

In [9]:
# Define hyperparameters that will be used in the Grid Search
baseline_parameters = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'gamma': [0.01, 0.1, 1.0, 'auto', 'scale'],
    'degree': np.arange(10),
}

In [10]:
svm_baseline = SVC(random_state=47)

In [11]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVC(C=10.0, degree=0, gamma=0.01, kernel='linear', random_state=47)
Accuracy:  0.8529411764705882


0.8529411764705882

### Random projections ECOC

In [12]:
encoding = np.eye(num_classes)
encoding

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [13]:
ecoc_proj_params = {
    'kernel': ['rbf'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
    'delta': [0.001, 0.01, 0.1, 1.0],
    'num_invariants': np.arange(1, num_dimensions + 1),
    'normalize_projections': [False, True],
}

In [14]:
ecoc_projections = SVMRandomInvariantsECOC(
    encoding,
    invariant_type=InvariantTypes.PROJECTION,
    random_state=47,
)

In [15]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.1, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        normalize_projections=True, num_invariants=12,
                        random_state=47)
Accuracy:  0.8088235294117647


0.8088235294117647

### Random hyperplanes ECOC

In [16]:
ecoc_hyper_params = {
    'kernel': ['rbf'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
    'delta': [0.001, 0.01, 0.1, 1.0],
    'num_invariants': np.arange(1, num_dimensions + 1),
}

In [17]:
ecoc_hyperplanes = SVMRandomInvariantsECOC(
    encoding,
    invariant_type=InvariantTypes.HYPERPLANE,
    random_state=47,
)

In [18]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=1.0,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=13, random_state=47)
Accuracy:  0.8529411764705882


0.8529411764705882

## Experiment 2: Using 50% of the data

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=47)

In [20]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVC(C=10.0, degree=0, gamma=0.01, kernel='linear', random_state=47)
Accuracy:  0.8452380952380952


0.8452380952380952

In [21]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.1, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        normalize_projections=True, num_invariants=9,
                        random_state=47)
Accuracy:  0.7857142857142857


0.7857142857142857

In [22]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=4, random_state=47)
Accuracy:  0.7797619047619048


0.7797619047619048

## Experiment 3: Using 30% of the data

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=47)

In [24]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVC(C=10.0, degree=0, gamma=0.01, kernel='linear', random_state=47)
Accuracy:  0.8135593220338984


0.8135593220338984

In [25]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.001, num_invariants=14, random_state=47)
Accuracy:  0.7669491525423728


0.7669491525423728

In [26]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=6, random_state=47)
Accuracy:  0.6949152542372882


0.6949152542372882

## Experiment 4: Using 20% of the data

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=47)

In [28]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVC(C=10.0, degree=0, gamma=0.01, kernel='linear', random_state=47)
Accuracy:  0.7992565055762082


0.7992565055762082

In [29]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        normalize_projections=True, num_invariants=13,
                        random_state=47)
Accuracy:  0.7955390334572491


0.7955390334572491

In [30]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=10.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=12, random_state=47)
Accuracy:  0.7174721189591078


0.7174721189591078

## Experiment 5: Using 10% of the data

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=47)

In [32]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVC(C=10.0, degree=0, gamma=0.01, kernel='linear', random_state=47)
Accuracy:  0.6765676567656765


0.6765676567656765

In [33]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.01, delta=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        num_invariants=1, random_state=47)
Accuracy:  0.49504950495049505


0.49504950495049505

In [34]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=7, random_state=47)
Accuracy:  0.5643564356435643


0.5643564356435643